# Irene City
From ancient Greek, *Εἰρήνη*: Peace.

This city finds the signal pulses within the waveforms produced by the
detector or by diomira in the case of Monte Carlo data.

This includes a number of tasks:
- Remove the signal-derivative effect of the PMT waveforms.
- Calibrate PMTs and produced a PMT-summed waveform.
- Remove the baseline from the SiPM waveforms and calibrate them.
- Apply a threshold to the PMT-summed waveform.
- Find pulses in the PMT-summed waveform.
- Match the time window of the PMT pulse with those in the SiPMs.
- Build the PMap object.

## How to use

In your shell, execute
```bash
cd /path/to/IC/work/dir/
source manage.sh work_in_python_version 3.7
```
to activate the *IC-3.7-2020-06-16* conda environment.

Then you are able to run *Irene City* with
```bash
city irene invisible_cities/config/irene.conf
```
being *irene.conf* the configuration for that run.

## Data Transformations

Each of the steps that this algorithm follows are described here. The parameters used in
all python snippets are described in the  [**Keys** section](##keys).


#### Map function

The [`fl.map`](doc/dataflow/map.md) function is described in *doc/dataflow/map.md*
in this project.


### Raw WaveForm to Corrected WaveForm

The first wave transformation done in *Irene City* is from Raw Waveform
to Corrected Wafevorm. This is made by this snippet
```python
from invisible_cities.dataflow import dataflow as fl
from invisible_cities.cities.components import deconv_pmt

# Raw WaveForm to Corrected WaveForm
rwf_to_cwf = fl.map(
    deconv_pmt(detector_db, run_number, n_baseline),
    args = "pmt",
    out  = "cwf"
    )
```
where the `fl.map` function applies to the `deconv_pmt` function output, which is another
function.

#### Deconv_pmt function

This `deconv_pmt(args, kwargs)` function generates another function `foo(RWF)` configured
by the `args` and `kwargs` given. The returned function `foo(RWF)` calculates the corrected
waveform (CWF) from raw waveform (RWF) given.

The `deconv_pmt(args, kwargs)` function takes as arguments (`args`):
- `dbfile`: String with name of database. With those names is chosen the correspondent
`sqlite3` file from `invisible_cities/database/`:
    + "new": localdb.NEWDB.sqlite3
    + "demopp": localdb.DEMOPPDB.sqlite3
    + "next100": localdb.NEXT100DB.sqlite3
    + "felx100": localdb.FLEX100DB.sqlite3
- `run_number`: Integer which determines what run number read in the SQL query to the
database.
- `n_baseline`: Limit to read in columns from RWF (*Raw Wave Form*)

and as keyword arguments (`kwargs`):
- `selection`: If `None` (default) it takes only active and nonzero values from data in database,
else it is a list with desired indices of the corrected waveform (CWF).
- `pedestal_function`: By default it uses `calib_sensors_functions.means` as function to take
the pedestal
```python
CWF = pedestal_function(RWF[:, :n_baseline]) - RWF
```
#### Example: *Raw WaveForm to Corrected WaveForm*

We take in this example the database called `localdb.NEWDB.sqlite3`, the run number `0`, and
for a window of 800 microseconds a baseline number of `28000`:

In [20]:
from invisible_cities.cities.components import deconv_pmt

# Raw WaveForm to Corrected WaveForm
foo = deconv_pmt(
    dbfile     = "new",
    run_number =     0,
    n_baseline = 28000  # for a window of 800 mus
)

and we get the `foo(RWF)` function with this given configuration. Then, if we apply the
`dataflow.map` function  to it

In [21]:
from invisible_cities.dataflow import dataflow as fl

rwf_to_cwf = fl.map(foo, args = "pmt", out  = "cwf")

we get the `rwf_to_cwf` function.

In [22]:
print(type(rwf_to_cwf))
print(f"Function called {rwf_to_cwf.__name__}")

print(dir(rwf_to_cwf))

<class 'function'>
Function called map_loop
['__annotations__', '__call__', '__class__', '__closure__', '__code__', '__defaults__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get__', '__getattribute__', '__globals__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__kwdefaults__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__wrapped__']


### Corrected WaveForm to Calibrated Corrected WaveForm

The next wave transformation done in *Irene City*, consists in take the
previous Corrected Waveform and calibrate it to get the Calibrated
Corrected WaveForm.
```python
cwf_to_ccwf = fl.map(
    calibrate_pmts(detector_db, run_number, n_mau, thr_mau),
    args = "cwf",
    out  = ("ccwfs", "ccwfs_mau", "cwf_sum", "cwf_sum_mau")
    )
```
#### Calibrate_pmts function
This `calibrate_pmts(args)` function with four arguments:
- `detector_db`
- `run_number`
- `n_mau`
- `thr_mau`

returns another function `foo(cwf)` with only one argument, which is the
calibrated waveform (`cwf`). This `foo(cwf)` is called for PMT waveforms that
have already been baseline restored and pedestal subtracted. It computes the
calibrated waveforms and its sensor sum. It also computes the calibrated waveforms
and sensor sum for elements of the waveforms above some value (`thr_mau`) over
a MAU that follows the waveform. These are useful to suppress oscillatory noise
and thus can be applied for S1 searches

#### Example: *Corrected WaveForm to Calibrated Corrected WaveForm*

In [1]:
print("Write example...")

Write example...


### Find Where Waveform is Above Threshold
```python
zero_suppress = fl.map(
    zero_suppress_wfs(thr_csum_s1, thr_csum_s2),
    args = ("cwf_sum", "cwf_sum_mau"),
    out  = ("s1_indices", "s2_indices", "s2_energies")
    )
```
#### Zero_suppress_wfs function
This `zero_suppress_wfs(args)` function with two arguments:
- `thr_csum_s1`: Threshold 1
- `thr_csum_s2`: Threshold 2

returns another function `foo(ccwf_sum, ccwf_sum_mau)` with two arguments
- `ccwf_sum`: Corrected waveform sum
- `ccwf_sum_mau`: corrected waveform sum mau
which returns indices of energies over the given thresholds `thr_csum_s1`
and `thr_csum_s2`.

### Remove Baseline and Calibrate SiPMs

Blah blah blah...

```python
sipm_rwf_to_cal = fl.map(
    calibrate_sipms(detector_db, run_number, sipm_thr),
    item = "sipm"
    )
```
#### Calibrate_sipms function
This `calibrate_sipms(args)` function with three arguments:
- `detector_db`
- `run_number`
- `sipm_thr`

returns another function `foo(rwf)` with only one argument, which is the
raw waveform (`rwf`). It subtracts the baseline, calibrates waveforms to
pes and suppresses values below `thr` (in pes) following given `args` from
`calibrate_sipms(args)`.


## Keys

Let's see what the hack are those configuration keys.

### detector_db

String which always takes
```python
detector_db = "new"
```
Always means, in all configuration files:
```bash
> cat invisible_cities/config/*.conf | grep detector_db
detector_db = 'new'
detector_db = 'new'
detector_db = 'new'
detector_db = 'new'
detector_db = 'new'
detector_db = 'new'
detector_db = 'new'
detector_db = 'new'
detector_db = 'new'
detector_db = 'new'
detector_db = 'new'
detector_db = 'new'
```

Why?

### thr_sipm_type
String to choose how to set the threshold. There are two options for this parameter:
- `"common"`: the threshold is a value in pes.
- `"individual"`: the threshold is a percentual value.

and you can write both in uppercase and lowercase.




## Input parameters
files_in
file_out
compression
event_range
print_mod
detector_db
run_number
n_baseline
n_mau
thr_mau
thr_sipm
thr_sipm_type
s1_lmin
s1_lmax
s1_tmin
s1_tmax
s1_rebin_stride
s1_stride
thr_csum_s1
s2_lmin
s2_lmax
s2_tmin
s2_tmax
s2_rebin_stride
s2_stride
thr_csum_s2
thr_sipm_s2
pmt_samp_wid=25*units.ns
sipm_samp_wid=1*units.mus